In [1]:
shell = get_ipython()
shell

In [2]:
kernel = shell.kernel
kernel

In [3]:
captured_events = []

def execute_request(stream, ident, parent):
    "Overwrite function to store the stream / ident /parent instead of calling kernel.execute_request"
    captured_events.append((stream, ident, parent))
    
def comm_msg(stream, ident, parent):
    "Overwrite function to add a logging (print) msg when comm_msg events come through"
    print("in comm_msg")
    return kernel.comm_manager.comm_msg(stream, ident, parent)
    
def start_capturing():
    "Overwrite the kernel shell handlers"
    kernel.shell_handlers['execute_request'] = execute_request
    kernel.shell_handlers['comm_msg'] = comm_msg
    
def stop_capturing():
    "rever the kernel shell handler functions to their defaults"
    kernel.shell_handlers['execute_request'] = kernel.execute_request
    kernel.shell_handlers['comm_msg'] = kernel.comm_manager.comm_msg

In [4]:
import ipywidgets as widgets
from IPython.display import display
import time
import zmq

w = widgets.Dropdown(options=['', 'foo', 'bar'])

start_capturing()
display(w)
while True:
    if w.value:
        print("widget value changed: breaking from loop")
        break # user changed the value
    
    for stream in kernel.shell_streams:
        stream.flush(zmq.POLLIN, 1)
        stream.flush(zmq.POLLOUT)
        
    time.sleep(.01)

stop_capturing()

Dropdown(options=('', 'foo', 'bar'), value='')

in comm_msg
widget value changed: breaking from loop


In [7]:
# run this while the last cell is blocking
print(w.value)

In [8]:
# run this while the last cell is blocking
print("hello world")

In [5]:
# don't run this until you changed the widget dropdown above..
captured_events

[(<zmq.eventloop.zmqstream.ZMQStream at 0x21850c54780>,
  [b'E4EEF76C98104DCA95D0F6471CBC9608'],
  {'buffers': [],
   'content': {'allow_stdin': True,
    'code': '# run this while the last cell is blocking\nprint(w.value)',
    'silent': False,
    'stop_on_error': True,
    'store_history': True,
    'user_expressions': {}},
   'header': {'date': datetime.datetime(2018, 2, 23, 18, 21, 18, 71000, tzinfo=tzlocal()),
    'msg_id': 'CD03C1CB802B4D8083FF8C1366466DBA',
    'msg_type': 'execute_request',
    'session': 'E4EEF76C98104DCA95D0F6471CBC9608',
    'username': 'username',
    'version': '5.0'},
   'metadata': {},
   'msg_id': 'CD03C1CB802B4D8083FF8C1366466DBA',
   'msg_type': 'execute_request',
   'parent_header': {}}),
 (<zmq.eventloop.zmqstream.ZMQStream at 0x21850c54780>,
  [b'E4EEF76C98104DCA95D0F6471CBC9608'],
  {'buffers': [],
   'content': {'allow_stdin': True,
    'code': '# run this while the last cell is blocking\nprint("hello world")',
    'silent': False,
    'stop_on_

In [9]:
for stream, ident, parent in captured_events:
    kernel.execute_request(stream, ident, parent)

foo
ERROR! Session/line number was not unique in database. History logging moved to new session 245
hello world
